# Check on the accuracy of the calculation of the QOCT gradients: Lindblad equation

In [ ]:
try:
    get_ipython
    isnotebook = True
except:
    isnotebook = False

import os
import sys
import numpy as np
import scipy as sp
import matplotlib
if not isnotebook:
    matplotlib.use('Agg')
import matplotlib.pyplot as plt
import time
from qutip import *
from qutip_qip.operations import toffoli

In [ ]:
import qocttools
import qocttools.models.GdW30 as GdW30
import qocttools.hamiltonians as hamiltonians
import qocttools.math_extra as math_extra
import qocttools.pulses as pulses
import qocttools.qoct as qoct
import qocttools.solvers as solvers
import qocttools.target as target

In [ ]:
qocttools.about()

In [ ]:
data = []

# Introduction

In order to calculate the optimal pulse that induces a given reaction in a quantum system, one defines a function of that pulse that must be optimized. One important ingredient for the optimization is derivative of this function with respect to the control parameters that define the pulse. In this script we check that this gradient or derivative is calculated correctly, for the case in which the Lindblad equation is used for the propagation of the density matrix.

# Model

The model is defined by the Hamiltonian:

\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
where the time-independent part is given by:
\begin{equation}
        \hat{H}_0 = D\bigg[\hat{S}_z^2 - \frac{1}{3}S(S + 1)\bigg] + E[\hat{S}_x^2 - \hat{S}_y^2] - g\mu_B\hat{\vec{S}}\cdot\vec{H}
\end{equation}
and the time-dependent part is:
\begin{equation}
        \hat{H}(t) = \hat{H}_0 + f(t)\hat{V}
\end{equation}
The perturbation is a magnetic field:
\begin{equation}
        \hat{V} = -g\mu_B\hat{\vec{S}}\cdot\vec{H}_m 
\end{equation}

In this case:

* $S = 7/2$

* $D$ = 1281 MHz

* $E$ = 294 MHz

* $\vec{H} = (0.15, 0.0, 0.0)$ T

* $\vec{H}_m = (0, 0.001, 0.0)$ T

In [ ]:
S = 7/2 # spin
E = 294 # value in MHz
D = 1281 # value in MHz
dim = int(2*S + 1) #matrix dim

In [ ]:
H = np.array([0.15, 0, 0.0], dtype = float) #magnetic field in T
H_m = np.array([0, 0.001, 0], dtype = float) #only in presence of perturbation (T)
H0 = GdW30.hGdW30(D, E, H)
V = GdW30.vGdW30(H_m)

In [ ]:
eigenvalues, eigenstates = H0.eigenstates()
#fileio.qsave(eigenstates, "eigenstates")
eigenstates = fileio.qload("eigenstates")

In [ ]:
# In principle, we could just transform V with the recently obtained eigenstates. Unfortunately, that
# would make the test results different in different computers, as the eigenstates can have different phases.
H0 = H0.transform(eigenstates) - eigenvalues[0]
V = V.transform(eigenstates)

In [ ]:
target_level = 1
P0 = fock_dm(dim, 0)
P1 = fock_dm(dim, target_level)

In [ ]:
Sx = jmat(S, "x")
Sy = jmat(S, "y")
Sz = jmat(S, "z")
Sp = (Sx + (1j*Sy))
Sm = (Sx - (1j*Sy))

In [ ]:
w = np.zeros(dim-1)
taui = np.zeros(dim-1)
for i in range(dim-1):
    w[i] = eigenvalues[i+1] - eigenvalues[i]
    taui[i] = 2.0*np.pi/w[i]
    print("Transition {:d}: w = {:f} MHz, tau = {:f} ns".format(i, w[i], 1000.0*taui[i]/(2.0*np.pi)))

# Time array definition

In [ ]:
T = 5*taui[0]
print("T = {:f} us*2*pi = {:f} ns".format(T, 1000*T/(2.0*np.pi)))
time = math_extra.timegrid(H0, T, 4.0)
print('# Time steps =', time.shape[0])

# Initial state

In [ ]:
state_0 = basis(dim, 0)
rho0 = state_0 * state_0.dag()

# Control function

The control function is parametrized with the Fourier expansion as follow:
\begin{equation}
    f(u, t) = \frac{1}{\sqrt{T}}u_0 + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k}\cos(\omega_kt) + \frac{2}{\sqrt{T}}\sum_{k = 1}^{M}u_{2k + 1}\sin(\omega_kt),
\end{equation}
where $u_0\dots u_{2M + 1}$ are the control parameters. This way, we can compute the derivate respect any control parameter as
\begin{equation}
    \frac{\partial f}{\partial u_m}(u, t) = f(e_m, t),
\end{equation}
where $e_m$ is the set of parameters where all of them are zero except the m-th ane, that is equal to one.

This pulse parametrization is included in the typical_pulses.py file as pulse class.

In [ ]:
M = 10

omega = np.zeros(M+1)
omega[0] = 0.0
for k in range(1, M+1):
    omega[k] = (2.0*np.pi/T) * k
    print("omega[{:d}] = {:f} MHz".format(k, omega[k]))

# u = 1.0*np.random.rand((2*M + 1))
u = np.zeros(2*M+1)
u[2] = 1.0
u[3] = 1.0
f = pulses.pulse("fourier", T, u = u)

In [ ]:
fig, ax = plt.subplots()

ax.plot(time * 1000/(2.0*np.pi), f.fu(time, u))
ax.set_xlabel("Time (ns)")
ax.set_xlim(left = 0.0, right = time[-1]*1000/(2.0*np.pi))
ax.set_ylabel("f(t) (mT)")
if isnotebook:
    plt.show()
else:
    fig.savefig("pulse.pdf")

# Propagation with the reference pulse (Schrödinger equation)

First, we do a normal propagation for the pure system using Schrödinger equation.

In [ ]:
result = solvers.solve('cfmagnus4', hamiltonians.hamiltonian(H0, V), f, state_0, time,
                       returnQoutput = False,
                       interaction_picture = False)

In [ ]:
p0 = np.zeros(time.shape[0])
p1 = np.zeros(time.shape[0])
for j in range(time.shape[0]):
    p0[j] = expect(P0, Qobj(result[j]))
    p1[j] = expect(P1, Qobj(result[j]))

In [ ]:
data.append(p1[-1])

In [ ]:
fig, ax = plt.subplots()

ax.plot(time, p0)
ax.plot(time, p1)

if isnotebook:
    plt.show()

# Propagation with the reference pulse (Lindblad equation)

Now, we will do a propagation adding dissipation, using Lindbad's equation. First, we define one "collapse" or Lindblad operator.

In [ ]:
A = 5.0*Sp

Now we are going to solve the equation twice, for checking purposes: once, using the qutip native propagator, and once using CFM4.

In [ ]:
result_good = solvers.solve('sesolve', hamiltonians.hamiltonian(H0, V, [A]), f, rho0, time,
                            returnQoutput = True,
                            interaction_picture = False)

In [ ]:
result = solvers.solve('cfmagnus4', hamiltonians.hamiltonian(H0, V, [A]), f, rho0, time,
                       returnQoutput = False,
                       interaction_picture = False)

In [ ]:
p0d_good = np.zeros(time.shape[0])
p1d_good = np.zeros(time.shape[0])
for j in range(time.shape[0]):
    p0d_good[j] = expect(P0, result_good[j])
    p1d_good[j] = expect(P1, result_good[j])

In [ ]:
p0d = np.zeros(time.shape[0])
p1d = np.zeros(time.shape[0])
for j in range(time.shape[0]):
    p0d[j] = expect(P0, Qobj(result[j]))
    p1d[j] = expect(P1, Qobj(result[j]))

In [ ]:
data.append(p1d[-1])

# Propagation with the reference pulse (Lindblad; superoperator representation)

Now, we repeat the propagation using the Lindblad equation. However, now we will use the "superoperator" or "Liouville space" representation, i.e. we will cast the densities into vectors, and the superoperators that act on the densities will be cast into matrices.

Convention:

* `X` will be the state or operator in the normal representation.
* `X_` will be the state or operator in Liouville representation.
* `X__` will be the staste or operrator in Liouville representation, although pretending it is just a normal vector or operator.

In [ ]:
rho0_ = operator_to_vector(rho0)
P0_ = operator_to_vector(P0)
P1_ = operator_to_vector(P1)
H0_ = (spre(H0)-spost(H0))
V_ = (spre(V)-spost(V))

Although qutip has a function that explicitly constructs the Liouvillian, we will explicitly build it here in order to compare it. The Liouvillian defined as:
\begin{equation}
\mathcal{L}(\rho) = -i[H_0,\rho] + A\rho A^\dagger - \frac{1}{2}\lbrace A^\dagger A, \rho\rbrace
\end{equation}
corresponds, in Liouville space representation, to:
\begin{equation}
L = -i I \otimes H_0 + i H_0 \otimes I + (A^\dagger)^T \otimes A
-\frac{1}{2} I \otimes A^\dagger A - \frac{1}{2} (A^\dagger A)^T \otimes I
\end{equation}

In [ ]:
Liouvillian = - 1j * tensor(qeye(dim), H0) + 1j * tensor(H0.trans(), qeye(dim)) \
              + tensor( (A.dag()).trans(), A) \
              - 0.5 * tensor( qeye(dim), A.dag()*A ) - 0.5 * tensor( (A.dag()*A).trans(), qeye(dim) )

The Liouvillian that propagates the costate is different (and in fact, we cannot build it with the `liouville` function of qutip:
\begin{equation}
L_{\rm costate} = -i L^\dagger = -i I \otimes H_0 + i H_0 \otimes I - A^T \otimes A^\dagger
+ \frac{1}{2} I \otimes A^\dagger A + \frac{1}{2} (A^\dagger A)^T \otimes I
\end{equation}
It corresponds to:
\begin{equation}
\mathcal{L}_{\rm costate}(\rho) = -i[H_0,\rho] - A^\dagger\rho A + \frac{1}{2}\lbrace A^\dagger A, \rho\rbrace
\end{equation}

In [ ]:
Liouvillian_costate = - 1j * tensor(qeye(dim), H0) + 1j * tensor(H0.trans(), qeye(dim)) \
                      - tensor( A.trans(), A.dag()) \
                      + 0.5 * tensor( qeye(dim), A.dag()*A ) + 0.5 * tensor( (A.dag()*A).trans(), qeye(dim) )

The costate Liouvillian should be equal to minus the adjoint of the state Liouvillian. We compare that now that this is indeed the case.

In [ ]:
Liouvillian_costate2 = - Liouvillian.dag()
print(np.linalg.norm(Liouvillian_costate.full()-Liouvillian_costate2.full()))

Now we build the Liouvillian using the `liouvillian` qutip function. we compare it to the one that we used before.

In [ ]:
Liouvillian_superop = liouvillian(H0, [A])
print(np.linalg.norm(Liouvillian.full()-Liouvillian_superop.full()))

We may assign a Hamiltonian to the superator Liouvillian, in the sense that $L = -i H_0$.

In [ ]:
H0_ = 1j*Liouvillian_superop

In [ ]:
rho0__ = Qobj(rho0_.full())
P0__ = Qobj(P0_.full())
P1__ = Qobj(P1_.full())
H0__ = Qobj(H0_.full())
V__ = Qobj(V_.full())
rho0__ = Qobj(rho0_.full())

In [ ]:
result = solvers.solve('cfmagnus4', hamiltonians.hamiltonian(H0__, V__), f, rho0__, time,
                       returnQoutput = False,
                       interaction_picture = False)

In [ ]:
p1vd = np.zeros(time.shape[0])
for j in range(time.shape[0]):
    p1vd[j] = P1__.overlap(Qobj(result[j])).real

In [ ]:
data.append(P1__.overlap(Qobj(result[-1])).real)

In [ ]:
fig, ax = plt.subplots()

ax.plot(time, p1, label = 'without dissipation')
ax.plot(time, p1d_good, marker = '.', linewidth = 0, label = 'with dissipation, sesolve')
ax.plot(time, p1d, label = 'with dissipation, cfmagnus')
ax.plot(time, p1vd, label = 'with dissipation, vectorized representation', linewidth = 0.2)
ax.legend()

if isnotebook:
    plt.show()

# Gradient calculation for state-to-state transitions (vectorized representation)

Now we will check that the computation of the QOCT gradient is correct. First, we will use the Liouville space representation.

We will choose, as a target function:
\begin{equation}
F(\rho, u) = Tr[P_1 \rho] = \langle\langle P_1 \vert \rho\rangle\rangle
\end{equation}
where we use the $\langle\langle\cdot\vert\cdot\rangle\rangle$ notation for the inner product in Liouville space. Note that this functional is real due to the Hermitian character of the operators $\rho$ and $P_1$. However, in order to take functional derivatives, one must be careful, and consider that in fact we are using:
\begin{equation}
F(\rho, u) = \frac{1}{2}(Tr[P_1^\dagger \rho] + Tr[\rho^\dagger P_1]) = 
\frac{1}{2}(\langle\langle P_1 \vert \rho\rangle\rangle + \langle\langle \rho\vert P_1\rangle\rangle\,.
\end{equation}

In [ ]:
def Fpsi(rho, u):
    return (P1__.dag() * rho).real

def dFdpsi(rho, u):
    return 0.5 * P1__

def dFdu(u, m):
    return 0.0

In [ ]:
H__ = hamiltonians.hamiltonian(H0__, [V__])

In [ ]:
tg = target.Target('generic', Fyu = Fpsi, dFdy = dFdpsi)

In [ ]:
opt = qoct.Qoct(H__, T, time.shape[0], tg, f, rho0__,
                interaction_picture = False,
                solve_method = 'cfmagnus4')

First, let us check that the number matches the one that has previously calculated:

In [ ]:
print(opt.gfunc(u))

In [ ]:
derqoct, dernum, error, elapsed_time = opt.check_grad(u, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

In [ ]:
data.append(derqoct)

# Gradient calculation for state-to-state transitions ("normal" representation)

Now the same, except that we use the "normal" representation for the density matrices.

In [ ]:
def Fpsi(rho, u):
    return expect(P1, rho)

def dFdpsi(rho, u):
    return 0.5 * P1

In [ ]:
H = hamiltonians.hamiltonian(H0, [V], [A])

In [ ]:
tg = target.Target('expectationvalue', operator = P1)

In [ ]:
opt = qoct.Qoct(H, T, time.shape[0], tg, f, rho0,
                interaction_picture = False,
                solve_method = 'cfmagnus4')

In [ ]:
print(opt.gfunc(u))

In [ ]:
derqoct, dernum, error, elapsed_time = opt.check_grad(u, 3)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))

In [ ]:
data.append(derqoct)

# Gradient calculation, multitarget case

In [ ]:
# We choose a toffoli gate as our target, but it could be any.
U_target = Qobj(toffoli().data)

We will have N=8 initial states $|j\rangle$, whose corresponding target states will be $|j'\rangle=U|j\rangle$.

In [ ]:
ket_ini = []
ket_target = []
rho_ini = []
rho_target = []

for i in range (0, dim):
    ket_ini.append(basis(dim,i))
    ket_target.append(U_target*ket_ini[i])

    rho_ini.append(fock_dm(dim,i))
    rho_target.append(U_target.dag()*rho_ini[i]*U_target)
    
rho=rho_ini #This will be the array of states that we will propagate.

In [ ]:
tg = target.Target('expectationvalue', operator = rho_target)

opt = qoct.Qoct(H, T, time.shape[0], tg, f, rho_ini,
                interaction_picture = True,
                solve_method = 'cfmagnus4')

In [ ]:
derqoct, dernum, error, elapsed_time = opt.check_grad(u)
print("QOCT calculation: \t{}".format(derqoct))
print("Ridders calculation: \t{} +- {}".format(dernum, error))
data.append(derqoct)

# Datafile

In [ ]:
with open("data", "w") as f:
    for i in data:
        f.write("{:.14e}\n".format(i))